# Brain Tumor Detection using CNN (PyTorch)

Binary classification: Brain Tumor vs Healthy

**Dataset**: Brain MRI images  
**Goal**: Classify brain scans as tumor or healthy  
**Framework**: PyTorch

## 1. Import Libraries

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import os
import pathlib

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchsummary import summary

from sklearn.metrics import confusion_matrix, classification_report
from tqdm.notebook import tqdm
import itertools
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


## 2. Data Preparation

**Split Dataset**: Use `splitfolders` to split into train/val (80/20)  
**Data Augmentation**: RandomHorizontalFlip, RandomRotation, Resize, Normalize

In [ ]:
input_path = '../input/brain-tumor-mri-dataset/'
output_path = 'brain_tumor_splitted'

splitfolders.ratio(input_path, output=output_path, seed=42, ratio=(0.8, 0.2))
print("Data split completed!")

**Code Walkthrough:**
- `splitfolders.ratio()` splits the dataset into train (80%) and validation (20%)
- `seed=42` ensures reproducible splits
- Creates folder structure: `brain_tumor_splitted/train/` and `brain_tumor_splitted/val/`

### Define Data Transforms

Apply augmentation to training data and normalization to both train and validation.

In [2]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

**Code Walkthrough:**
- **Training transforms**: Resize to 224x224, flip horizontally (50% chance), rotate up to 10°, normalize
- **Validation transforms**: Only resize and normalize (no augmentation for validation)
- `Normalize(mean=[0.5], std=[0.5])` scales pixel values to [-1, 1] range

### Load Datasets

In [ ]:
train_path = 'brain_tumor_splitted/train'
val_path = 'brain_tumor_splitted/val'

train_set = ImageFolder(train_path, transform=train_transform)
val_set = ImageFolder(val_path, transform=val_transform)

print(f"Training samples: {len(train_set)}")
print(f"Validation samples: {len(val_set)}")
print(f"Classes: {train_set.classes}")

### Create DataLoaders

In [ ]:
batch_size = 64

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)

for X, y in train_loader:
    print(f"Batch shape: {X.shape}")  
    print(f"Label shape: {y.shape}")
    break

## 3. Define CNN Model

**Architecture**: 4 Convolutional blocks + 2 Fully Connected layers  
**Features**: Batch Normalization, Dropout, MaxPooling  
**Output**: Binary classification (Tumor vs Healthy)

In [5]:
class CNN_TUMOR(nn.Module):
    def __init__(self):
        super(CNN_TUMOR, self).__init__()
        
        # Conv Block 1: 1 -> 8 channels
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        
        # Conv Block 2: 8 -> 16 channels
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        
        # Conv Block 3: 16 -> 32 channels
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        
        # Conv Block 4: 32 -> 64 channels
        self.conv4 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        
        # Fully Connected layers
        # After 4 pooling layers: 224 -> 112 -> 56 -> 28 -> 14
        # Feature map size: 14x14x64 = 12544
        self.fc1 = nn.Linear(64 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 2)
        
    def forward(self, x):
        # Block 1
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        
        # Block 2
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        
        # Block 3
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        # Block 4
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        
        # Flatten
        x = x.view(-1, 64 * 14 * 14)
        
        # FC layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

model = CNN_TUMOR().to(device)
print(model)

CNN_TUMOR(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=12544, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)


**Model Architecture:**

| Layer | Input | Output | Operation |
|-------|-------|--------|-----------|
| Conv1 + Pool | 224×224×1 | 112×112×8 | Conv → BN → ReLU → MaxPool |
| Conv2 + Pool | 112×112×8 | 56×56×16 | Conv → BN → ReLU → MaxPool |
| Conv3 + Pool | 56×56×16 | 28×28×32 | Conv → BN → ReLU → MaxPool |
| Conv4 + Pool | 28×28×32 | 14×14×64 | Conv → BN → ReLU → MaxPool |
| Flatten | 14×14×64 | 12544 | Reshape to 1D |
| FC1 | 12544 | 128 | Linear → ReLU → Dropout(0.5) |
| FC2 | 128 | 2 | Linear (output logits) |

**Code Walkthrough:**
- **4 Conv blocks**: Each doubles the channels (1→8→16→32→64), halves spatial size with pooling
- **Batch Normalization**: Stabilizes training by normalizing each layer's output
- **Dropout (50%)**: Randomly drops neurons during training to prevent overfitting
- **Output**: 2 classes (0=Healthy, 1=Tumor)

### Model Summary (Architecture Visualization)

In [6]:
summary(model, input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 224, 224]              80
       BatchNorm2d-2          [-1, 8, 224, 224]              16
         MaxPool2d-3          [-1, 8, 112, 112]               0
            Conv2d-4         [-1, 16, 112, 112]           1,168
       BatchNorm2d-5         [-1, 16, 112, 112]              32
         MaxPool2d-6           [-1, 16, 56, 56]               0
            Conv2d-7           [-1, 32, 56, 56]           4,640
       BatchNorm2d-8           [-1, 32, 56, 56]              64
         MaxPool2d-9           [-1, 32, 28, 28]               0
           Conv2d-10           [-1, 64, 28, 28]          18,496
      BatchNorm2d-11           [-1, 64, 28, 28]             128
        MaxPool2d-12           [-1, 64, 14, 14]               0
           Linear-13                  [-1, 128]       1,605,760
          Dropout-14                  [

**Summary shows:**
- Layer-by-layer architecture
- Output shape at each layer
- Number of parameters (weights and biases)
- Total parameters and model size

## 4. Training Setup

**Loss Function**: CrossEntropyLoss (for classification)  
**Optimizer**: Adam with learning rate 0.001  
**Scheduler**: ReduceLROnPlateau (reduces LR when validation loss plateaus)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 5. Training Loop

In [ ]:
epochs = 30
train_losses = []
val_losses = []

for epoch in range(epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Calculate average training loss
    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)
    
    # Validation phase
    model.eval()
    val_running_loss = 0.0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()
    
    # Calculate average validation loss
    val_loss = val_running_loss / len(val_loader)
    val_losses.append(val_loss)
    
    # Print progress every 5 epochs
    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

print('Finished Training')

**Training Loop Breakdown:**

**Outer loop (epochs)**: Runs 30 times over the entire dataset

**Training phase**:
1. `model.train()`: Sets model to training mode (enables dropout, batch norm)
2. `optimizer.zero_grad()`: Clears old gradients from previous iteration
3. `outputs = model(images)`: Forward pass → computes predictions
4. `loss = criterion(outputs, labels)`: Computes cross-entropy loss
5. `loss.backward()`: Backpropagation → computes gradients
6. `optimizer.step()`: Updates weights using gradients

**Validation phase**:
- `model.eval()`: Sets model to evaluation mode (disables dropout, batch norm)
- `with torch.no_grad()`: Disables gradient calculation for faster computation
- Computes validation loss to track model performance on unseen data

**Print**: Shows training and validation loss every 5 epochs

### Plot Training History

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

## 6. Model Evaluation

Evaluate model performance using confusion matrix and classification report.

In [ ]:
def get_predictions(model, dataloader):
    """Get all predictions and true labels"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    return np.array(all_preds), np.array(all_labels)

y_pred, y_true = get_predictions(model, val_loader)

### Confusion Matrix

In [ ]:
def plot_confusion_matrix(cm, classes):
    """Plot confusion matrix heatmap"""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, train_set.classes)

**Understanding the Confusion Matrix:**
- **Top-left (True Negatives)**: Correctly predicted as Healthy
- **Top-right (False Positives)**: Incorrectly predicted as Tumor (false alarm)
- **Bottom-left (False Negatives)**: Incorrectly predicted as Healthy (missed tumor)
- **Bottom-right (True Positives)**: Correctly predicted as Tumor

### Classification Report

In [ ]:
print(classification_report(y_true, y_pred, target_names=train_set.classes))

**Metrics Explained:**
- **Precision**: Of all predicted tumors, how many are actually tumors? (TP / (TP + FP))
- **Recall**: Of all actual tumors, how many did we detect? (TP / (TP + FN))
- **F1-Score**: Harmonic mean of precision and recall (balanced metric)
- **Support**: Number of samples in each class

### Visualize Predictions

Show sample images with their predicted and actual labels.

In [ ]:
# Get a batch of validation images
dataiter = iter(val_loader)
images, labels = next(dataiter)

# Get predictions
model.eval()
with torch.no_grad():
    images = images.to(device)
    outputs = model(images)
    _, predictions = torch.max(outputs, 1)

# Move to CPU for visualization
images = images.cpu()
predictions = predictions.cpu()

# Plot 8 sample images
fig, axes = plt.subplots(2, 4, figsize=(15, 8))
axes = axes.ravel()

for i in range(8):
    # Convert from normalized tensor to image
    img = images[i].squeeze()  # Remove channel dimension for grayscale
    
    # Denormalize: reverse the normalization (mean=0.5, std=0.5)
    img = img * 0.5 + 0.5
    
    axes[i].imshow(img, cmap='gray')
    axes[i].set_title(f'True: {train_set.classes[labels[i]]}\nPred: {train_set.classes[predictions[i]]}',
                     color='green' if labels[i] == predictions[i] else 'red')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

**Visualization shows:**
- **Green titles**: Correct predictions
- **Red titles**: Incorrect predictions (misclassifications)
- Images are denormalized from [-1, 1] back to [0, 1] for proper display

## 7. Save Model

In [ ]:
torch.save(model.state_dict(), 'brain_tumor_cnn.pth')
print("Model saved successfully!")